In [24]:
# install needed package

#install.packages("XLConnect")
#install.packages("rvest")
#install.packages("rjson")
#install.packages("httr")
#install.packages("stringr")
#install.packages("dbplyr")
#install.packages("dplyr")
#install.packages("RSQLite")

In [25]:
# loading needed library
library(rvest)
library(rjson)
library(httr)
library(stringr)
library(dbplyr)
library(dplyr)
library(RSQLite)
library(DBI)
library(XLConnect)
library(tidyverse)
library(lubridate)

In [26]:
# scrape data from Misnitry of Transport - Vehicle kilometres travelled by region (billion km)
# https://www.transport.govt.nz/statistics-and-insights/road-transport/sheet/vehicle-kms-travelled-vkt

host_url <- "https://public.tableau.com"
path <- "/views/RoadTransport_0/RD086-Vehiclekilometrestravelledbyregionbillionkm?"

body <- read_html(modify_url(host_url, 
                             path = path, 
                             query = ":embed=yes&:showVizHome=no&:tabs=no&:toolbar=yes"
))

data <- body %>% 
  html_nodes("textarea#tsConfigContainer") %>% 
  html_text()
json <- fromJSON(data)

url <- modify_url(host_url, path = paste(json$vizql_root, "/bootstrapSession/sessions/", json$sessionid, sep =""))

resp <- POST(url, body = list(sheet_id = json$sheetId), encode = "form")
data <- content(resp, "text")


# The scraping code below credited for Doughey https://stackoverflow.com/questions/64256354/scraping-a-websites-tableau-dashboard-using-r

extract <- str_match(data, "\\d+;(\\{.*\\})\\d+;(\\{.*\\})")
data <- fromJSON(extract[1,3])
worksheet = names(data$secondaryInfo$presModelMap$vizData$presModelHolder$genPresModelMapPresModel$presModelMap)
columnsData <- data$secondaryInfo$presModelMap$vizData$presModelHolder$genPresModelMapPresModel$presModelMap[[worksheet]]$presModelHolder$genVizDataPresModel$paneColumnsData

i <- 1
result <- list();
for(t in columnsData$vizDataColumns){
  if (is.null(t[["fieldCaption"]]) == FALSE) {
    paneIndex <- t$paneIndices
    columnIndex <- t$columnIndices
    if (length(t$paneIndices) > 1){
      paneIndex <- t$paneIndices[1]
    }
    if (length(t$columnIndices) > 1){
      columnIndex <- t$columnIndices[1]
    }
    result[[i]] <- list(
      fieldCaption = t[["fieldCaption"]], 
      valueIndices = columnsData$paneColumnsList[[paneIndex + 1]]$vizPaneColumns[[columnIndex + 1]]$valueIndices,
      aliasIndices = columnsData$paneColumnsList[[paneIndex + 1]]$vizPaneColumns[[columnIndex + 1]]$aliasIndices, 
      dataType = t[["dataType"]],
      stringsAsFactors = FALSE
    )
    i <- i + 1
  }
}
dataFull = data$secondaryInfo$presModelMap$dataDictionary$presModelHolder$genDataDictionaryPresModel$dataSegments[["0"]]$dataColumns

cstring <- list();
for(t in dataFull) {
  if(t$dataType == "cstring"){
    cstring <- t
    break
  }
}
data_index <- 1
name_index <- 1
frameData <-  list()
frameNames <- c()
for(t in dataFull) {
  for(index in result) {
    if (t$dataType == index["dataType"]){
      if (length(index$valueIndices) > 0) {
        j <- 1
        vector <- character(length(index$valueIndices))
        for (it in index$valueIndices){
          vector[j] <- t$dataValues[it+1]
          j <- j + 1
        }
        frameData[[data_index]] <- vector
        frameNames[[name_index]] <- paste(index$fieldCaption, "value", sep="-")
        data_index <- data_index + 1
        name_index <- name_index + 1
      }
      if (length(index$aliasIndices) > 0) {
        j <- 1
        vector <- character(length(index$aliasIndices))
        for (it in index$aliasIndices){
          if (it >= 0){
            vector[j] <- t$dataValues[it+1]
          } else {
            vector[j] <- cstring$dataValues[abs(it)]
          }
          j <- j + 1
        }
        frameData[[data_index]] <- vector
        frameNames[[name_index]] <- paste(index$fieldCaption, "alias", sep="-")
        data_index <- data_index + 1
        name_index <- name_index + 1
      }
    }
  }
}

df <- NULL
lengthList <- c()
for(i in 1:length(frameNames)){
  lengthList[i] <- length(frameData[[i]])
}
max <- max(lengthList)
for(i in 1:length(frameNames)){
  if (length(frameData[[i]]) < max){
    len <- length(frameData[[i]])
    frameData[[i]][(len+1):max]<-""
  }
  df[frameNames[[i]]] <- frameData[i]
}
options(width = 1200)
df <- as.data.frame(df, stringsAsFactors = FALSE)
print(df)

    Measure.Values.value Measure.Values.alias       Region.alias                                    Measure.Names.value Measure.Names.alias
1       11.1307165891137                11.13      Other Regions [federated.0npm3nk137dwgj1bs20161izsoga].[sum:2020:qk]                2020
2       11.4119746208798                11.41      Other Regions [federated.0npm3nk137dwgj1bs20161izsoga].[sum:2019:qk]                2019
3        11.365312090245                11.37      Other Regions [federated.0npm3nk137dwgj1bs20161izsoga].[sum:2018:qk]                2018
4       11.0152014879107                11.02      Other Regions [federated.0npm3nk137dwgj1bs20161izsoga].[sum:2017:qk]                2017
5       10.6201532333023                10.62      Other Regions [federated.0npm3nk137dwgj1bs20161izsoga].[sum:2016:qk]                2016
6       10.1251600666945                10.13      Other Regions [federated.0npm3nk137dwgj1bs20161izsoga].[sum:2015:qk]                2015
7        9.756896382

In [27]:
# wrangling the VKT dataframe

VKTdf = subset(df,select = -c(Measure.Values.alias,Measure.Names.value))
colnames(VKTdf) = c("VKT","region","year")
VKTdf$VKT = as.numeric(VKTdf$VKT)
VKTdf$year = as.numeric(VKTdf$year)
VKTdf = VKTdf[,c(2,3,1)]

In [28]:
# download Population estimated file from Environment Canterbury Regional Council
popu_url = "https://www.ecan.govt.nz/document/download/?uri=4571117"
download.file(popu_url,"population.xlsx")

# load xlsx file to dataframe
population_df = readWorksheetFromFile("population.xlsx", 
                                    sheet=2, 
                                    startRow = 3, endRow = 22,
                                    startCol =1, endCol = 22)


#wrangling the data frame

# change column names
colnames(population_df) = c('region','2001','2002','2003','2004','2005','2006','2007','2008',
'2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021')

# convert dataframe to long format
population_df = population_df %>%
                pivot_longer(cols =c('2001','2002','2003','2004','2005','2006','2007','2008',
                                    '2009','2010','2011','2012','2013','2014','2015','2016',
                                    '2017','2018','2019','2020','2021'),
                            names_to = 'year',
                            values_to ='population'
                            )
# change year datatype from character to Date
population_df$year = year(as.Date(population_df$year, format = "%Y"))

# remove the word Region from the region name
population_df$region<-gsub("Region","",as.character(population_df$region))

In [29]:
# Load the Crashes Analysis System data to dataframe
CAS <- read.csv("CAS.csv")

#splited to relational data frame
locationCols <- c("OBJECTID", "X", "Y", "areaUnitID", "crashDirectionDescription", "crashLocation1", "crashLocation2", "crashSHDescription", "meshblockId", "region", "tlaId", "tlaName", "urban")
detailCols <- c("OBJECTID", "crashFinancialYear", "crashSeverity", "crashYear", "directionRoleDescription", "fatalCount", "holiday", "minorInjuryCount", "region","seriousInjuryCount")
weatherCols <- c("OBJECTID", "light", "weatherA", "weatherB")
vehicleCols <- c("OBJECTID", "bicycle", "bus", "carStationWagon", "moped", "motorcycle", "otherVehicleType", "pedestrian", "schoolBus", "suv", "taxi", "truck", "unknownVehicleType", "vanOrUtility", "vehicle")
environmentCols <- c("OBJECTID", "advisorySpeed", "bridge", "cliffBank", "debris", "ditch", "fence", "flatHill", "guardRail", "houseOrBuilding", "kerb", "NumberOfLanes", "objectThrownOrDropped", "otherObject", "overBank", "parkedVehicle", "phoneBoxEtc", "postOrPole", "roadCharacter", "roadLane", "roadSurface", "roadworks", "slipOrFlood", "speedLimit", "strayAnimal", "streetLight", "temporarySpeedLimit", "trafficControl", "trafficIsland", "trafficSign", "train", "tree", "waterRiver")

#create dataframes
location <- CAS[locationCols]
crashesDetail <- CAS[detailCols]
environmentalCon <- CAS[weatherCols]
vehicleInvolved <- CAS[vehicleCols]
surroundingCon <- CAS[environmentCols]

In [30]:
# create an SQLite database 
db = dbConnect(RSQLite::SQLite(), "CAS.db")

In [31]:
# create VKT table frame
dbExecute(db, 
            "CREATE TABLE VKT (
            region varchar(20) NOT NULL,
            year INTEGER NOT NULL,
            VKT float,
            PRIMARY KEY (region, year)
            );"
        )
# write dataframe to db VKT table        
dbWriteTable(db, "VKT", VKTdf, append = T)

[1] 0

In [32]:
# create Population table frame
dbExecute(db, 
            "CREATE TABLE population (
            region varchar(20) NOT NULL,
            year INTEGER NOT NULL,
            population INTEGER NOT NULL,
            PRIMARY KEY (region, year)
            );"
        )
# write dataframe to db VKT table        
dbWriteTable(db, "population", population_df, append = T)

[1] 0

In [33]:
# create crashesDetail table frame
dbExecute(db, 
            "CREATE TABLE crashesDetail (
            OBJECTID INTERGER NOT NULL,
            crashFinancialYear varchar(10), 
            crashSeverity varchar(20), 
            crashYear INTERGER NOT NULL, 
            directionRoleDescription varchar(10), 
            fatalCount INTERGER, 
            holiday varchar(30), 
            minorInjuryCount INTERGER, 
            region varchar(20),
            seriousInjuryCount INTERGER,
            PRIMARY KEY (OBJECTID)
            );"
        )
# write dataframe to db VKT table        
dbWriteTable(db, "crashesDetail", crashesDetail, append = T)

[1] 0

In [34]:
# create environmentalCon table frame
dbExecute(db, 
            "CREATE TABLE environmentalCon (
            OBJECTID INTERGER NOT NULL,
            light varchar(20), 
            weatherA varchar(20),
            weatherB varchar(20),
            PRIMARY KEY (OBJECTID)
            );"
        )
# write dataframe to db environmentalCon table
dbWriteTable(db, "environmentalCon", environmentalCon, append = T)

[1] 0

In [35]:
# create vehicleInvolved table frame
dbExecute(db, 
            "CREATE TABLE vehicleInvolved (
            OBJECTID INTERGER NOT NULL,
            bicycle INTERGER,
            bus INTERGER,
            carStationWagon INTERGER,
            moped INTERGER,
            motorcycle INTERGER,
            otherVehicleType INTERGER,
            pedestrian INTERGER,
            schoolBus INTERGER,
            suv INTERGER,
            taxi INTERGER,
            truck INTERGER,
            unknownVehicleType INTERGER,
            vanOrUtility INTERGER,
            vehicle INTERGER,
            PRIMARY KEY (OBJECTID)
            );"
        )
# write dataframe to db vehicleInvolved table
dbWriteTable(db, "vehicleInvolved", vehicleInvolved, append = T)

[1] 0

In [36]:
# create surroundingCon table frame
dbExecute(db, 
            "CREATE TABLE surroundingCon (
            OBJECTID INTERGER NOT NULL,
            advisorySpeed INTERGER, 
            bridge INTERGER,
            cliffBank INTERGER,
            debris INTERGER,
            ditch INTERGER,
            fence INTERGER,
            flatHill varchar(20),
            guardRail INTERGER,
            houseOrBuilding INTERGER,
            kerb INTERGER,
            NumberOfLanes INTERGER,
            objectThrownOrDropped INTERGER,
            otherObject INTERGER,
            overBank INTERGER,
            parkedVehicle INTERGER,
            phoneBoxEtc INTERGER,
            postOrPole INTERGER,
            roadCharacter varchar(20),
            roadLane varchar(20), 
            roadSurface varchar(20),
            roadworks INTERGER,
            slipOrFlood INTERGER,
            speedLimit INTERGER,
            strayAnimal INTERGER,
            streetLight varchar(10), 
            temporarySpeedLimit INTERGER,
            trafficControl varchar(30),
            trafficIsland INTERGER,
            trafficSign INTERGER,
            train INTERGER,
            tree INTERGER,
            waterRiver INTERGER,
            PRIMARY KEY (OBJECTID)
            );"
        )
# write dataframe to db surroundingCon table
dbWriteTable(db, "surroundingCon", surroundingCon, append = T)

[1] 0

In [37]:
# remove temp files
unlink("CAS.csv")
unlink("population.xlsx")

In [38]:
# disconnect from database
 dbDisconnect(db)